# Mad1 inference new objective function 

Set the hyperparameters to $S=2,R=1$.

In [1]:
hypars = [2,1]; # [S,R]
# pf defined for the priors (see SI).
pf = (hypars[1]-1)/2

0.5

Gene number is 3 for mad1endog.

In [2]:
gn = 3;

Import the relevant packages and simulations.

In [3]:
using Random, Plots, LaTeXStrings, Distributions, StatsBase, DataFrames, CSV, Plots.PlotMeasures, Revise, GpABC, JLD, FileIO

In [4]:
Plots.theme(:dao)
Plots.gr_cbar_width[] = 0.01;

In [5]:
Revise.track("/home/jamesh/GitHub/volume-inference-take-2/SSA-basic.jl")
include("/home/jamesh/GitHub/volume-inference-take-2/SSA-basic.jl")
using .SSAnonVD

Define a nice colorblind color scheme.

In [6]:
pal = cgrad(:tol_muted);
cpal = [pal[i] for i in 1/7:1/7:1.0]
population_colors = cpal

Import the data.

In [7]:
df_nuc = DataFrame(CSV.File("/home/jamesh/GitHub/volume-inference-take-2/data/nuclear-vol-corr.csv"));
df_cyto = DataFrame(CSV.File("/home/jamesh/GitHub/volume-inference-take-2/data/cyto-vol-corr.csv"));
delete!(df_nuc, 2);
delete!(df_cyto, 2);

In [8]:
gnames = df_cyto[!,"gene"]
Ns = df_nuc[!,"N"];
cyto_means = df_cyto[!,"Mean"];
nuc_FFs = df_nuc[!,"FF"];
cyto_FFs = df_cyto[!,"FF"];

In [9]:
gnames[gn]

"mad1endog"

Define the timescales necessary to predict the steady-state time. `sim_time` defines the point at which the steady state is assumed to be reached.

In [10]:
tdeg = 1;
texpt(kA,kB,kC,S,R) = 1/kC;
tinit(kA,kB,kC,S,R) = (1/kB)+(S-1)/kA;

In [11]:
function sim_time(pars)
    kA, kB, kC = pars;
    return 10*maximum([1,texpt(pars...,hypars...),tinit(pars...,hypars...)])
end

sim_time (generic function with 1 method)

Import the values from the bootstrapping for the distance function weighting. It is the inverse of these errors that multiples each square error (larger variance in the sample statistic gives it a lesser weighting in the objective function). 

In [12]:
# bootstrapping = DataFrame(CSV.File("/home/jamesh/GitHub/volume-inference-take-2/data/meanCyto_FF_withVar_forJames_230422.csv"))
# ro_df = DataFrame([[] for i in 1:length(names(bootstrapping))], names(bootstrapping)) # empty df
# for name in gnames
#     ind = findfirst(isequal(name), bootstrapping[!,:gene])
#     push!(ro_df, bootstrapping[ind,:])
# end
# save("./ABC-new/ABC-redo/bootstrapping.jld", "reordered-BS", ro_df)

In [13]:
bootstrapping=load("./ABC-new/ABC-redo/bootstrapping.jld")["reordered-BS"];

┌ Warning: type InlineStrings.String31 not present in workspace; reconstructing
└ @ JLD ~/.julia/packages/JLD/S6t6A/src/jld_types.jl:697
┌ Warning: type InlineStrings.String15 not present in workspace; reconstructing
└ @ JLD ~/.julia/packages/JLD/S6t6A/src/jld_types.jl:697


In [14]:
names(bootstrapping);

Take the bootstrapped sample variances (can now use the index `gn` defined above).

In [15]:
bs_cyto = bootstrapping[!,"varmeanRNA_Cytoplasm"][gn]
bs_ffn = bootstrapping[!,"varFF_Nucleus"][gn]
bs_ffc = bootstrapping[!,"varFF_Cytoplasm"][gn];
bs_vec = [bs_cyto, bs_ffn, bs_ffc]

3-element Vector{Float64}:
 0.00091527517086321
 0.00092299262167585
 0.00053918925784561

Define the distance function.

In [16]:
function new_distance(traj1::Matrix, traj2::Matrix)
    return sum([bs_vec[i]^(-1)*((traj1[i]-traj2[i])/traj2[i])^2 for i in 1:length(traj1)])
end
function new_distance(traj1::Vector, traj2::Vector)
    return sum([bs_vec[i]^(-1)*((traj1[i]-traj2[i])/traj2[i])^2 for i in 1:length(traj1)])
end

new_distance (generic function with 2 methods)

# Let's do the ABC!

In [17]:
gnames[gn]

"mad1endog"

In [18]:
function simulator_fn_3(pars)
    op = SSA(Ns[gn], pars, hypars, sim_time(pars), sim_time(pars));
    nuc_pop = op[1][:,2];
    cyto_pop = op[2][:,2];
    nuc_FF = var(nuc_pop)/mean(nuc_pop);
    cyto_mean = 2*mean(cyto_pop);
    cyto_FF = var(cyto_pop)/mean(cyto_pop);
    # return the moments of interest
    return reshape([cyto_mean, nuc_FF, cyto_FF],(3,1))
end

simulator_fn_3 (generic function with 1 method)

Let's look at the value of the objective function for something close to an optimum value. This informs what our threshold schedule for the ABC will be.

In [19]:
new_distance(simulator_fn_3([2.54,3.64,20.0]),reshape([cyto_means[gn],nuc_FFs[gn],cyto_FFs[gn]],(3,1)))

787.2327150938522

In [44]:
# ABC priors
priors_3 = [Uniform(pf*0.5,pf*10.5), Uniform(0.5, 10.5), Uniform(0.5,100.5)]

# number of particles
n_particles = 1000

# ABC hpars
threshold_schedule_3 = [200.0, 100.0, 50.0, 25.0, 20.0, 10.0, 5.0]

7-element Vector{Float64}:
 200.0
 100.0
  50.0
  25.0
  20.0
  10.0
   5.0

In [45]:
sim_abcsmc_res_3 = SimulatedABCSMC(reshape([cyto_means[gn],nuc_FFs[gn],cyto_FFs[gn]],(3,1)),
    simulator_fn_3,
    priors_3,
    threshold_schedule_3,
    n_particles; 
    max_iter=convert(Int, 1e5),
    distance_function = new_distance,
    write_progress=true);

[ Info: GpABC rejection simulation. ϵ = 200.0.
[ Info: GpABC rejection simulation. Accepted 28/1000 particles.
[ Info: GpABC rejection simulation. Accepted 56/2000 particles.
[ Info: GpABC rejection simulation. Accepted 83/3000 particles.


LoadError: InterruptException:

Quick look at the results...

In [46]:
plot(sim_abcsmc_res_3, population_colors=population_colors)

LoadError: UndefVarError: sim_abcsmc_res_3 not defined

Save these results using the JLD package.

In [47]:
save("./ABC-new/ABC-redo/$(gnames[gn])/S$(hypars[1])R$(hypars[2]).jld", "ABC", sim_abcsmc_res_3)

LoadError: UndefVarError: sim_abcsmc_res_3 not defined

Let's collect the important aspects of this ABC for analysis. We want:
- 1. The parameter values of the accepted particles.
- 2. The acceptance rate for the final threshold.

Note the different field names are: `n_accepted`, `n_tries`, `threshold_schedule`, `population`, `distances`, `weights`.

In [24]:
acc_rate = sim_abcsmc_res_3.n_accepted[end]/sim_abcsmc_res_3.n_tries[end]

LoadError: UndefVarError: sim_abcsmc_res_3 not defined

So the acceptance rate at the final threshold was around 0.2. Let's now look at the mean and standard deviations of the predicted values of $k_A$, $k_B$ and $k_C$ from the population.

In [25]:
pop = sim_abcsmc_res_3.population[end];

LoadError: UndefVarError: sim_abcsmc_res_3 not defined

In [26]:
kA = pop[:,1]; kB = pop[:,2]; kC = pop[:,3];

LoadError: UndefVarError: pop not defined

In [27]:
kAmed = median(kA)

LoadError: UndefVarError: kA not defined

In [28]:
kAstd = std(kA)

LoadError: UndefVarError: kA not defined

In [29]:
kBmed = median(kB)

LoadError: UndefVarError: kB not defined

In [30]:
kBstd = std(kB)

LoadError: UndefVarError: kB not defined

In [31]:
kCmed = median(kC)

LoadError: UndefVarError: kC not defined

In [32]:
kCstd = std(kC)

LoadError: UndefVarError: kC not defined

Let's make some nice plots of these (the three histograms followed by the joint uncertainty plots as a 2D hist).

In [33]:
h1 = stephist(kA, fill = true, fillalpha = 0.5, color = cpal[1], label = :none, norm = true, titlelocation = :left)
plot!(grid = false, minorgrid = false, legend = :topright, size = (450,400), xlabel = L"k_A", title = "(a)", ylims = [0,3.5])
plot!([kAmed,kAmed],[0,100], style = :dash, color = :black, label = L"$k_C$ median");

LoadError: UndefVarError: kA not defined

In [34]:
h2 = stephist(kB, fill = true, fillalpha = 0.5, color = cpal[2], label = :none, norm = true, titlelocation = :left)
plot!(grid = false, minorgrid = false, legend = :topright, size = (450,400), xlabel = L"k_B", title = "(b)", ylims = [0,0.3])
plot!([kBmed,kBmed],[0,1.2], style = :dash, color = :black, label = L"$k_B$ median");

LoadError: UndefVarError: kB not defined

In [35]:
h3 = stephist(kC, fill = true, fillalpha = 0.5, color = cpal[3], label = :none, norm = true, titlelocation = :left)
plot!(grid = false, minorgrid = false, legend = :topright, size = (450,400), xlabel = L"k_C", 
        title = "(c)", ylims = [0,0.025])
plot!([kCmed,kCmed],[0,2], style = :dash, color = :black, label = L"$k_C$ median");

LoadError: UndefVarError: kC not defined

In [36]:
c1 = plot(h1, h2, h3, layout = (1,3), size = (1000, 300), bottom_margin = 5mm, top_margin = 5mm);

LoadError: UndefVarError: h1 not defined

In [37]:
p1 = histogram2d(kA, kB, color=:tol_muted, show_empty_bins=true, titlelocation = :left, 
    bins = (collect(minimum(kA):(maximum(kA)-minimum(kA))/15:maximum(kA)), collect(minimum(kB):(maximum(kB)-minimum(kB))/15:maximum(kB))))
plot!(grid = false, minorgrid = false, legend = :topright, size = (450,400), 
    xlabel = L"k_A", ylabel = L"k_B", title = "(d)");

LoadError: UndefVarError: kA not defined

In [38]:
p2 = histogram2d(kA, kC, color=:tol_muted, show_empty_bins=true, titlelocation = :left, 
    bins = (collect(minimum(kA):(maximum(kA)-minimum(kA))/15:maximum(kA)), collect(minimum(kC):(maximum(kC)-minimum(kC))/15:maximum(kC))))
plot!(grid = false, minorgrid = false, legend = :topright, size = (450,400), 
    xlabel = L"k_A", ylabel = L"k_C", title = "(e)");

LoadError: UndefVarError: kA not defined

In [39]:
p3 = histogram2d(kB, kC, color=:tol_muted, show_empty_bins=true, titlelocation = :left, 
    bins = (collect(minimum(kB):(maximum(kB)-minimum(kB))/15:maximum(kB)), collect(minimum(kC):(maximum(kC)-minimum(kC))/15:maximum(kC))))
plot!(grid = false, minorgrid = false, legend = :topright, size = (450,400), 
    xlabel = L"k_B", ylabel = L"k_C", title = "(f)");

LoadError: UndefVarError: kB not defined

In [40]:
c2 = plot(p1, p2, p3, layout = (1,3), size = (1000, 300), bottom_margin = 5mm, top_margin = 5mm, left_margin = 5.0mm, right_margin = -10mm);

LoadError: UndefVarError: p1 not defined

In [41]:
plot(c1,c2, layout = (2,1), size = (1000,600), plot_title=L"%$(gnames[gn]): $S=R=3$")
# savefig("./ABC-new/figs/$(gnames[gn])/S$(hypars[1])R$(hypars[2]).pdf")

LoadError: UndefVarError: c1 not defined

Check if the moments replicate those from the original data.

In [42]:
[cyto_means[gn],nuc_FFs[gn],cyto_FFs[gn]]

3-element Vector{Float64}:
 1.74181548708104
 0.92893586062368
 0.619817985303977

In [43]:
mean([simulator_fn_3([kAmed,kBmed,kCmed]) for i in 1:100])

LoadError: UndefVarError: kAmed not defined

So, pretty good overall.